In [1]:
import pandas as pd
from itertools import product
from tqdm.notebook import tqdm

import sys, os

sys.path.append(os.path.abspath('..'))
%load_ext autoreload
%autoreload 2
from modules.storage import (
    get_demand_model_data,
    get_demand_orig_dest_model_data,
)
from modules.config import *
from modules.svm import *

In [2]:
all_possible_metas = [
    {'kernel': ['linear'], 'C': [0.1, 1, 10, 100],  'gamma': [-1],                          'degree': [-1],         'max_iter': [1000000]},
    {'kernel': ['rbf'],    'C': [0.1, 1, 10, 100],  'gamma': [0.1, 0.01, 0.001, 0.0001],    'degree': [-1],         'max_iter': [1000000]},
    {'kernel': ['poly'],   'C': [1, 10, 100, 1000], 'gamma': [-1],                          'degree': [2, 3, 4, 5], 'max_iter': [1000000]}
]

In [3]:
execute_stage(
    'first_stage',
    SVM_FIRST_STAGE_DEMAND_RESULTS_PATH,
    SVM_SECOND_STAGE_DEMAND_RESULTS_PATH,
    TUNE_H3_RESOLUTION,
    TUNE_TIME_INTERVAL_LENGTH,
    all_possible_metas,
    get_demand_model_data,
)

  0%|          | 0/3 [00:00<?, ?it/s]

h3: 8 | t:6 | - linear ✓
h3: 8 | t:6 | - rbf ✓


In [ ]:
results = pd.read_parquet(SVM_FIRST_STAGE_DEMAND_RESULTS_PATH)
results.sort_values(by=['mean_test_score'], ascending=False).head(2)

,iter,n_resources,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_kernel,param_max_iter,params,...,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score,n_iter,h3_res,time_interval_length,param_gamma,param_degree
51,1,9999,94.646476,4.913808,7.762252,0.495021,10.0,poly,1000000,"{'C': 10.0, 'degree': 2.0, 'gamma': None, 'ker...",...,-1.417860,-1.401450,-1.362995,-1.389943,0.018701,0,8,6,NaN,2.0
29,2,29997,492.754425,15.538668,111.743918,4.525014,1.0,rbf,1000000,"{'C': 1.0, 'degree': None, 'gamma': 0.1, 'kern...",...,-1.083101,-1.062449,-1.060248,-1.082855,0.026092,0,8,6,0.1,NaN


In [ ]:
results.sort_values(by='mean_test_score', ascending=False).groupby('param_kernel')[[
    'mean_test_score',
    'param_C',
    'param_degree',
    'param_gamma',
]].first()

,mean_test_score,param_C,param_degree,param_gamma
param_kernel,,,,
linear,-1.650525,0.1,NaN,NaN
poly,-1.463993,10.0,2.0,NaN
rbf,-1.467642,1.0,NaN,0.1


In [ ]:
results.groupby('param_kernel').mean_fit_time.sum() / 60

param_kernel
linear    31.433408
poly      34.511246
rbf       24.907050
Name: mean_fit_time, dtype: float64

In [ ]:
resolutions = list(product(PREDICTIVE_H3_RESOLUTIONS, CALC_TIME_INTERVAL_LENGTHS))
resolutions = resolutions + (ADDITIONAL_PREDICTIVE_RESOLUTIONS)
resolutions

[(7, 1), (7, 2), (7, 6), (7, 24), (8, 1), (8, 2), (8, 6), (8, 24), (9, 24)]

In [ ]:
for h3_res, time_interval_length in tqdm(resolutions):
    execute_stage(
        'second_stage',
        SVM_FIRST_STAGE_DEMAND_RESULTS_PATH,
        SVM_SECOND_STAGE_DEMAND_RESULTS_PATH,
        h3_res,
        time_interval_length,
        all_possible_metas,
        get_demand_model_data,
    )

  0%|          | 0/9 [00:00<?, ?it/s]

In [ ]:
results = pd.read_parquet(SVM_SECOND_STAGE_DEMAND_RESULTS_PATH)
results.sort_values(by=['mean_test_score'], ascending=False)

,iter,n_resources,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_gamma,param_kernel,param_max_iter,...,std_train_score,n_iter,h3_res,time_interval_length,test_mse,test_rmse,test_mae,test_non_zero_mape,test_zero_accuracy,param_degree
4,0,30000,262.145430,34.717682,60.153046,3.134454,100.0,0.1,rbf,1000000,...,0.003112,48807,8,1,0.359630,0.599692,0.363834,0.306788,0.998884,NaN
8,0,30000,541.641010,23.750486,122.150627,11.681470,100.0,0.1,rbf,1000000,...,0.006336,35656,9,24,0.383422,0.619211,0.320453,0.274703,0.998812,NaN
5,0,30000,240.854028,14.668349,53.294168,4.500610,100.0,0.1,rbf,1000000,...,0.005110,50555,8,2,0.683795,0.826919,0.476454,0.373026,0.997403,NaN
6,0,30000,236.306019,14.349750,49.945286,4.529440,100.0,0.1,rbf,1000000,...,0.026270,53708,8,6,1.908995,1.381664,0.763860,0.496193,0.995245,NaN
7,0,30000,238.910257,10.407333,51.801087,2.396608,100.0,0.1,rbf,1000000,...,0.082838,46834,8,24,6.699621,2.588363,1.387782,0.628280,0.998315,NaN
0,0,30000,133.982361,5.805195,26.131435,1.382498,100.0,0.1,rbf,1000000,...,0.055206,63318,7,1,7.284058,2.698899,1.566463,0.687987,0.993165,NaN
1,0,30000,123.734570,5.820446,23.595334,1.267245,100.0,0.1,rbf,1000000,...,0.347656,69807,7,2,19.568643,4.423646,2.445755,0.783617,0.992675,NaN
2,0,30000,118.448948,4.517039,23.960149,0.652030,100.0,0.1,rbf,1000000,...,1.831843,65815,7,6,89.240300,9.446708,4.768756,0.872286,0.995018,NaN
3,0,23710,69.155549,2.869557,14.840478,0.364704,100.0,0.1,rbf,1000000,...,11.737005,33745,7,24,594.101570,24.374199,11.170370,0.921739,0.999325,NaN


In [ ]:
results_first_stage = pd.read_parquet(SVM_FIRST_STAGE_DEMAND_RESULTS_PATH)
results_second_stage = pd.read_parquet(SVM_SECOND_STAGE_DEMAND_RESULTS_PATH)
results_first_stage.to_parquet(os.path.join(REPO_DATA_DIR_PATH, f"{str(SVM_MAX_TRAIN_SET_SIZE)}_svm_first_stage_demand_results.parquet"))
results_second_stage.to_parquet(os.path.join(REPO_DATA_DIR_PATH, f"{str(SVM_MAX_TRAIN_SET_SIZE)}_svm_second_stage_demand_results.parquet"))

In [ ]:
execute_stage(
    'second_stage',
    SVM_FIRST_STAGE_DEMAND_RESULTS_PATH,
    SVM_SECOND_STAGE_DEMAND_RESULTS_PATH,
    int(f"{h3_res}{h3_res}"),
    time_interval_length,
    all_possible_metas,
    lambda _, __, : get_demand_orig_dest_model_data(),
)